# Решение задачи классификации текстов по сантименту при помощи нейронных сетей

### Эпиграф

Будучи глубоко неудовлетворен теми результатами, которых позволяла достичь линейная и логистическая регрессия, я решил отложить сдачу финального проекта до тех пор, пока не обучусь нейросетям. Т.к. в настоящей специализации они проходились, я считаю такое решение задачи нейросетями совершенно легитимным.


In [ ]:
!pip install nltk
!pip install tensorflow

In [1]:
#1. Импортируем общие библиотеки.
import pandas as pd
import numpy as np
import nltk
nltk.download("punkt")
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(42)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rookie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#2. Импортируем тензорфлоу и керас
#Рассчитано на TF 2.0^

import tensorflow as tf
print(tf.__version__)
import keras
import keras.backend as K
import keras.layers as L
import tensorflow.compat.v1 as v1
np.random.seed(42)

#Функция очистки сессии
def reset_tf_session():
    curr_session = v1.get_default_session()
    if curr_session is not None:
        curr_session.close()
    # reset graph
    K.clear_session()
    # create new session
    config = v1.ConfigProto()
    config.gpu_options.allow_growth = True
    s = v1.InteractiveSession(config=config)
    v1.keras.backend.set_session
    return s

reset_tf_session()

2.3.1


Идея состоит в следующем. Целый ряд испытаний с LSTM показал, что учитывать последовательность слов или н-грамм не имеет никакого смысла, поэтому я использую подход Bag of Words. Т.к. используемое количество токенов в обучающей выборке невелико (~4000), в качестве векторизатора используем TfIdfVectorizer

1. Т.к. данные довольно хорошо предобработаны (например, все приведено к нижнему регистру, пунктуация уже отделена пробелами), разбиваем исходные тексты на слова простым методом .split() и обучаем CountVectorizer на исходных текстах - cоздаем Bag of Words, подготавливаем функцию to_matrix, которая предобрабатывает данные "от и до".
3. Подаем векторизованные тексты на вход нейронной сети и обучаем ее.
4. Достаточно 10-20 эпох, разбиваем выборку на трейн и валидацию так же, как вся обучающая выборка соотносится с тестовой 2000/500.

In [3]:
from nltk import bigrams, ngrams, everygrams
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

#3. Готовим первичные выборки, разбиваем тексты на n-граммы от одного слова до четырех с помощью nltk.everygrams, делаем словарь
df = pd.read_csv("products_sentiment_train.tsv", sep='\t', header=None)
df.columns = ["text", "label"]

df.head()


,text,label
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [4]:
#X = [x[0] for x in df[["text"]].values.tolist()]   #Выборка, где элементы - строки
#Y = [y[0] for y in df[["label"]].values.tolist()]

X = df["text"].values
Y = df["label"].values

vectorizer = TfidfVectorizer()
vectorizer.fit(X)

#Векторизуем тексты обученным векторизатором
def to_matrix(texts):
    return np.array([vectorizer.transform([text]).toarray().squeeze() for text in texts])
#На входе массив (None,) -> на выходе массив (None, N_DIM)

a = to_matrix(X[0:1])
#Длина строк one-hot-encoding, выдаваемого CountVectorizer'ом
N_DIM = a.shape[1]

len(X), len(Y), X[0], N_DIM

(2000, 2000, '2 . take around 10,000 640x480 pictures .', 3973)

In [6]:
vectorizer.transform([X[0]]).toarray().squeeze()

array([0.        , 0.50364999, 0.37302423, ..., 0.        , 0.        ,
       0.        ])

In [61]:
#5.1. Строим модель нейронной сети

#Используем оптимизатор Adam и бинарную кроссэнтропию в качестве функции потерь
def build_model():
    #Input - входные данные, шейп (None, N_DIM) означает, что мы можем принимать батч произвольной длины,
    #со вторым измерением равным полученному из CountVectorizera
    X = L.Input(batch_input_shape=(None, N_DIM))
    #Три полносвязных слоя для лучшего обучения
    d = L.Dense(1024, activation="relu")(X)
    d = L.Dropout(0.2)(d)
    d = L.Dense(32, activation="relu")(X)
    d = L.Dropout(0.2)(d)
    Y = L.Dense(1, activation="sigmoid")(d)
    return keras.models.Model(inputs=(X), outputs=Y)

t_model = build_model()
opt = keras.optimizers.Adam(lr=0.001)
t_model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])
t_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3973)]            0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                127168    
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 127,201
Trainable params: 127,201
Non-trainable params: 0
_________________________________________________________________


In [62]:
N_BATCH = 4

#Тренируем модель
def train_model(xt, yt, 
                xv, yv,
                initial_epoch=0, n_epochs=10):
    t_model.fit(to_matrix(xt), np.array(yt), epochs=n_epochs, 
                shuffle=True,
                validation_data=(to_matrix(xv), np.array(yv)),
                batch_size=N_BATCH, initial_epoch=initial_epoch)
    
reset_tf_session()

opt = keras.optimizers.Adam(lr=0.0001)
t_model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

#Xt, Xv, Yt, Yv = train_test_split(X, Y, train_size=0.75)
#train_model(Xt, Yt, Xv, Yv, 0, 5)
for i in range():
    Xt, Xv, Yt, Yv = train_test_split(X, Y, train_size=0.75)
    train_model(Xt, Yt, Xv, Yv, initial_epoch=i, n_epochs=i+1)

375/375 [==============================] - 1s 3ms/step - loss: 0.6845 - accuracy: 0.6247 - val_loss: 0.6754 - val_accuracy: 0.6400
Epoch 2/2
375/375 [==============================] - 1s 3ms/step - loss: 0.6668 - accuracy: 0.6313 - val_loss: 0.6523 - val_accuracy: 0.6540
Epoch 3/3
375/375 [==============================] - 1s 3ms/step - loss: 0.6451 - accuracy: 0.6427 - val_loss: 0.6432 - val_accuracy: 0.6200
Epoch 4/4
375/375 [==============================] - 1s 3ms/step - loss: 0.6323 - accuracy: 0.6287 - val_loss: 0.6128 - val_accuracy: 0.6620
Epoch 5/5
375/375 [==============================] - 1s 3ms/step - loss: 0.6165 - accuracy: 0.6293 - val_loss: 0.5959 - val_accuracy: 0.6600
Epoch 6/6
375/375 [==============================] - 1s 3ms/step - loss: 0.5853 - accuracy: 0.6547 - val_loss: 0.6120 - val_accuracy: 0.5900
Epoch 7/7
375/375 [==============================] - 1s 3ms/step - loss: 0.5715 - accuracy: 0.6580 - val_loss: 0.5705 - val_accuracy: 0.6420
Epoch 8/8
375/375 [====

In [54]:
df = pd.read_csv("products_sentiment_test.tsv", sep='\t')
X_test = [x[0] for x in df[["text"]].values.tolist()]

Y_pred = [0 if y < 0.5 else 1 for y in t_model.predict(to_matrix(X_test))]
len(X_test), len(Y_pred)

(500, 500)

После обучения в течение 50 эпох точность составила 1.0 на обучаемой выборке. Применим к тестовой выборке.

In [55]:
df = pd.DataFrame()
df["y"] = Y_pred
df.head()

df.to_csv("kaggle_submission.csv", sep=',', index_label="Id")

In [ ]:
from IPython.display import Image

Image(filename = 'screen.png')